In [22]:
import yaml
from jinja2 import Template
from glob import glob
import os.path
import os


def get_template(path):
    with open(path) as f:
        return Template(f.read())
    
def get_yaml(path):
    with open(path) as f:
        return yaml.load(f, yaml.Loader)
    
def write_template(path, template, d):
    with open(path, 'w') as f:
        f.write(template.render(**d))
        
html_path = './html'
plan_path = './plan.yaml'
meal_template = get_template('./templates/meal.html')
plan_template = get_template('./templates/index.html')

In [23]:
plan = get_yaml(plan_path)

In [24]:
meal_data = {}
for meal_path in glob('./meals/*.yaml'):
    print(meal_path)
    meal_filename = os.path.split(meal_path)[1]
    meal_name = os.path.splitext(meal_filename)[0]
    meal_html_name = meal_name + '.html'
    meal_html_path = os.path.join(html_path, meal_html_name)
    meal_data[meal_name] = get_yaml(meal_path)
    write_template(meal_html_path, meal_template, meal_data[meal_name])

./meals/breakfast-bars.yaml
./meals/breakfast-for-dinner.yaml
./meals/apple-pie-porridge.yaml
./meals/black-bean-soup.yaml
./meals/mushroom-and-wild-rice-soup.yaml
./meals/mac-and-cheese.yaml
./meals/red-beans-and-rice.yaml


In [25]:
plan

{'monday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'},
 'tuesday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'},
 'wednesday': {'breakfast': 'breakfast-bars',
  'dinner': 'breakfast-for-dinner'},
 'thursday': {'breakfast': 'breakfast-bars', 'dinner': 'red-beans-and-rice'},
 'friday': {'breakfast': 'apple-pie-porridge',
  'dinner': 'mushroom-and-wild-rice-soup'},
 'saturday': {'breakfast': 'breakfast-bars', 'dinner': 'mac-and-cheese'},
 'sunday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'}}

In [26]:
plan_html_path = os.path.join(html_path, 'index.html')
d = {'meal_data':meal_data, 'plan': plan}
write_template(plan_html_path, plan_template, d)

In [27]:
plan

{'monday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'},
 'tuesday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'},
 'wednesday': {'breakfast': 'breakfast-bars',
  'dinner': 'breakfast-for-dinner'},
 'thursday': {'breakfast': 'breakfast-bars', 'dinner': 'red-beans-and-rice'},
 'friday': {'breakfast': 'apple-pie-porridge',
  'dinner': 'mushroom-and-wild-rice-soup'},
 'saturday': {'breakfast': 'breakfast-bars', 'dinner': 'mac-and-cheese'},
 'sunday': {'breakfast': 'breakfast-bars', 'dinner': 'black-bean-soup'}}

In [28]:
meal_data['breakfast-bars']['name']

'Breakfast bars'